# Timing attack to guess password from extremely unsecure server

In [92]:
import timeit
import itertools
import random
import string
import numpy as np
from IPython.display import display, clear_output

In [41]:
password_database = {'user1': 'blablabla', 'user2' :'1iaj-1aXr-lu0j-9zia', 'user3': 'fjs9328rughenvml'}
allowed_chars = string.ascii_lowercase + string.ascii_uppercase + '1234567890' + '-+/_.:?! '

In [42]:
# The weak server
def check_password(username, password):
    if username not in password_database.keys():
        return False
    actual = password_database[username]
    if len(password) != len(actual):
        return False

    for i in range(len(actual)):
        if password[i] != actual[i]:
            return False
    return True

In [43]:
def random_string(length):
    return ''.join(random.choices(allowed_chars, k=length))

In [44]:
def crack_password_length(username, max_length=32):
    trials = 1000
    timings = np.empty(max_length)
    for i in range(max_length):
        time_i = timeit.repeat(stmt='check_password(username, x)',
                      setup=f'username={username!r};x=random_string({i!r})',
                      globals=globals(), 
                      number=trials,
                      repeat=10)
        timings[i] = min(time_i)
    return int(np.argmax(timings))

In [102]:
def crack_password(username, length):
    guess = random_string(length)
    counter = itertools.count()
    trials = 1000
    while True:
        j = 0
        i = next(counter) % length
        for c in allowed_chars:
            j+=1
            new = guess[:i] + c + guess[i+1:]
            timings_new = timeit.repeat(stmt='check_password(username, x)',
                              setup=f'username={username!r};x={new!r}',
                              globals=globals(), 
                              number=trials,
                              repeat=10)
            timings_guess = timeit.repeat(stmt='check_password(username, x)',
                              setup=f'username={username!r};x={guess!r}',
                              globals=globals(), 
                              number=trials,
                              repeat=10)
            if check_password(username, new):
                clear_output(wait=True)
                display(new)
                return new, j
            if min(timings_new) > min(timings_guess):
                r = random.randint(0,10)
                guess = new
                clear_output(wait=True)
                display(guess)

In [103]:
def main():
    username = 'user2'
    length = crack_password_length(username)
    print(f"using most likely length {length}")
    input("hit enter to CRACK...")
    password, j = crack_password(username, length)
    print(f"password cracked in {str(j)} iterations:'{password}'")

In [104]:
main()

'1iaj-1aXr-lu0j-9zia'

password cracked in 1 iterations:'1iaj-1aXr-lu0j-9zia'
